<a href="https://colab.research.google.com/github/Yuhan0524/TDAforcell/blob/main/Cell_Image_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
class Real_Distance:
  def real_euclidean_distance(point1, point2):
      return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
  def calculate_distance_matrix(point_cloud):
      num_points = len(point_cloud)
      distance_matrix = np.zeros((num_points, num_points))

      for i in range(num_points):
          for j in range(i + 1, num_points):
              distance = Real_Distance.real_euclidean_distance(point_cloud[i][:2], point_cloud[j][:2])
              distance_matrix[i, j] = distance
              distance_matrix[j, i] = distance

      return distance_matrix

In [ ]:
class Find_Pairs:
    @staticmethod
    def find_pairs_within_epsilon(distance_matrix, epsilon):
        num_points = distance_matrix.shape[0]
        indices = np.arange(num_points)
        labels_dict = {}

        for i in range(num_points):
            mask = distance_matrix[i, i+1:] <= epsilon
            labels = indices[i+1:][mask].tolist()
            if labels:
                labels_dict[i] = labels

        return labels_dict

In [ ]:
class Get_Weight:
    @staticmethod
    def calculate_distances(point_cloud):
        num_points = len(point_cloud)
        distances = np.zeros((num_points, num_points))

        for i in range(num_points):
            for j in range(i + 1, num_points):
                distance = np.linalg.norm(np.array(point_cloud[i][2:-1]) - np.array(point_cloud[j][2:-1]))
                distances[i, j] = distance
                distances[j, i] = distance
        return distances

    def rescale_distance(point_cloud, numberoffeature):
        distance_matrix = Get_Weight.calculate_distances(point_cloud, numberoffeature)
        max_distance = np.max(distance_matrix)
        rescaled_matrix = distance_matrix - max_distance
        rescaled_matrix /= -max_distance
        rescaled_matrix = 2 * rescaled_matrix - 1
        return rescaled_matrix

In [ ]:
class Graph:
    @staticmethod
    def generate_adjacency_matrix(connected_nodes, distance_matrix):
        num_nodes = len(distance_matrix)
        adjacency_matrix = np.full((num_nodes, num_nodes), np.inf)

        for node, connected in connected_nodes.items():
            adjacency_matrix[node, connected] = distance_matrix[node, connected]
            adjacency_matrix[connected, node] = distance_matrix[node, connected]

        np.fill_diagonal(adjacency_matrix, 0)
        return adjacency_matrix

In [ ]:
class Choose_Connection:
  def connectunderthreshold(adjacency_matrix, threshold, a):
    labels_dict = {}
    num_points = adjacency_matrix.shape[0]
    for i in range(num_points):
      labels = []
      for j in range(i + 1, num_points):
        if a == "above":
          if 100000 > adjacency_matrix[i, j] >= threshold: #choose the one above the threshold
            labels.append(j)
        if a == "below":
          if adjacency_matrix[i,j] <= threshold: #choose the one below the threshold
            labels.append(j)
      if labels:
        labels_dict[i] = labels

    return labels_dict

In [ ]:
import pandas as pd
import numpy as np

data_1 = pd.read_csv("/content/drive/MyDrive/GIS/A_cell_table_size_normalized.csv")
data1 = data_1[data_1['fov'] == '2022-12-03T15-18-19_BIGFISH1_pA_s1_R10C5']
X1_coor = data1[["centroid-0", "centroid-1"]].values
overlapping_markers = data1.columns[2:48]
Y1 = data1[overlapping_markers].values
point_cloud_ini = np.concatenate((X1_coor, Y1), axis=1)
point_cloud = [(*row, i) for i, row in enumerate(point_cloud_ini)]

In [ ]:
#generate the distance matrix first
distance_matrix = Real_Distance.calculate_distance_matrix(point_cloud)
#dissimilarity_matrix = Get_Weight.rescale_distance(point_cloud)
dissimilarity_matrix = Get_Weight.calculate_distances(point_cloud)

In [ ]:
#change the epsilon value accordingly, which represents the maximum distance between 2 connected cells
#if a fully connected image is required (all the cells within the distance are connected, without filtering the dissimilarity), use the next line only
labels_within_epsilon = Find_Pairs.find_pairs_within_epsilon(distance_matrix, 60)

#If a filtration for dissimialrity is needed, please un-command the following

#connected_nodes = labels_within_epsilon
#dissimilarity_matrix = np.array(dissimilarity_matrix)
#adjacency_matrix = Graph.generate_adjacency_matrix(connected_nodes, dissimilarity_matrix)
#newlabel = Choose_Connection.connectunderthreshold(adjacency_matrix, 0.99, "above") #whether to filter "above"/"below" the threshold can change in the Choose_Connection.connectunderthreshold function directly

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import numpy as np

line_segments = []
points = []
point_to_segments = {}

#if a visualization including filtration of disimilarity is needed, use the newlabel one and command the line "for node, connected in labels_within_epsilon.items():"; else just directly run the code
#for node, connected in newlabel.items():
for node, connected in labels_within_epsilon.items():
    coor = point_cloud[node][:2]
    for connected_node in connected:
        connected_coor = point_cloud[connected_node][:2]
        line_segments.append([coor, connected_coor])
        points.append(coor)
        points.append(connected_coor)
        if coor not in point_to_segments:
            point_to_segments[coor] = []
        point_to_segments[coor].append(connected_coor)

line_segments_1 = np.array(line_segments)
points = np.array(points)

triangles = []
for segment in line_segments:
    p1, p2 = segment

    if p1 in point_to_segments and p2 in point_to_segments:
        connected_segments_p1 = point_to_segments[p1]
        connected_segments_p2 = point_to_segments[p2]

        for seg_p1 in connected_segments_p1:
            for seg_p2 in connected_segments_p2:
                if seg_p1[0] == seg_p2[0]:
                    triangles.append([p1, p2, seg_p1])

poly_collection = collections.PolyCollection(triangles, facecolors='orange')

plt.figure(figsize=(24, 24))

plt.plot(line_segments_1[:, :, 0].T, line_segments_1[:, :, 1].T, 'k-')
plt.scatter(points[:, 0], points[:, 1], color='blue', marker='o')
plt.gca().add_collection(poly_collection)

plt.xlim(0, 1024)  # Adjust the x-axis limits as needed
plt.ylim(0, 1024)  # Adjust the y-axis limits as needed
plt.xlabel('X')
plt.ylabel('Y')

plt.show()